In [1]:
import gym
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression
import os
import numpy as np

In [2]:
class agent():
    def __init__(self, name, feature, label):
        self.name = name
        self.feature = np.array(feature)
        self.label = np.array(label)
        
        self.feature_size = self.feature.shape
        self.feature_size = self.feature_size[1]
        self.label_size = self.label.shape
        self.label_size = self.label_size[1]
        self.model = self.create_model()
        
        self.main_dir = self.name
        self.model_dir = self.main_dir + '/model_save'
        
        if not os.path.exists(self.main_dir):
            os.makedirs(self.main_dir)
            
    def saver(self):
        self.model.save(self.model_dir)
    
    def restore(self):
        self.model.load(self.model_dir)
        
    def train_for(self, epoch):
        self.model.fit({'input': self.feature}, {'targets': self.label}, n_epoch=epoch, snapshot_step=500, show_metric=True, run_id='openai_learning')
    
    def create_model(self):
        keep = 0.8
        LR = 1e-3
        network = input_data(shape=[None, self.feature_size], name='input')
        network = fully_connected(network, 128, activation='relu', name = 'hidden_1')
        network = dropout(network,keep)
        network = fully_connected(network, 256, activation='relu', name = 'hidden_2')
        network = dropout(network,keep)
        network = fully_connected(network, 512, activation='relu', name = 'hidden_3')
        network = dropout(network,keep)
        network = fully_connected(network, 256, activation='relu', name = 'hidden_4')
        network = dropout(network,keep)
        network = fully_connected(network, 128, activation='relu', name = 'hidden_5')
        network = dropout(network,keep)
        network = fully_connected(network, self.label_size, activation='softmax', name = 'softmax')
        network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
        model = tflearn.DNN(network, tensorboard_verbose=3)
        return model
        

In [3]:
def convert_labels(labels):
    training_data = []
    for data in labels:
        if data == 0:
            output = [1,0,0]
        if data == 1:
            output = [0,1,0]
        if data == 2:
            output = [0,0,1]
        training_data.append(output)
    return training_data

In [4]:
if __name__ == '__main__':
    from sklearn.datasets import load_iris

    iris = load_iris()
    features = iris.data
    labels = iris.target
    labels = convert_labels(labels)
    a = agent(feature=features, label=labels, name='iris')
    a.train_for(100)
    a.saver()

Training Step: 299  | total loss: 0.24531 | time: 0.054s
| Adam | epoch: 100 | loss: 0.24531 - acc: 0.9412 -- iter: 128/150
Training Step: 300  | total loss: 0.60235 | time: 0.083s
| Adam | epoch: 100 | loss: 0.60235 - acc: 0.8743 -- iter: 150/150
--
INFO:tensorflow:/home/hedonist/Documents/general_dnn/iris/model_save is not in all_model_checkpoint_paths. Manually adding it.


[2017-07-11 13:26:22,012] /home/hedonist/Documents/general_dnn/iris/model_save is not in all_model_checkpoint_paths. Manually adding it.


In [4]:
def runmnist():
    import database

    a = agent(feature=database.train_x, label=database.train_y, name='mnist')
    a.train_for(5)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Training Step: 4299  | total loss: 0.18977 | time: 34.329s
| Adam | epoch: 005 | loss: 0.18977 - acc: 0.9625 -- iter: 54976/55000
Training Step: 4300  | total loss: 0.18015 | time: 34.367s
| Adam | epoch: 005 | loss: 0.18015 - acc: 0.9625 -- iter: 55000/55000
--
